In [45]:
# tf.keras.Model如何最终到其持有的各个变量的
import tensorflow as tf
import numpy as np
class MyModelWithoutBuild(tf.keras.Model):
    def __init__(self):
        super(MyModelWithoutBuild,self).__init__()
        self.dense_layer1=tf.keras.layers.Dense(units=2)
        self.dense_layer2=tf.keras.layers.Dense(units=2)
        self.a_useless_var=tf.Variable([1.1])

    def call(self, inputs, training=None, mask=None):
        output=self.dense_layer1(inputs)
        output=self.dense_layer2(output)
        return output


input_batch=np.reshape(np.random.random(100),newshape=(10,10)).astype(np.float32)

print("该模型没有复写build方法，只覆盖了call方法")
my_model=MyModelWithoutBuild()
print("调用call之前，可以看到trainable_variables只有一个a_useless_var")
print(my_model.trainable_variables)
my_model(input_batch)
print("\n调用call之后，可以看到trainable_variables新增了各个layer的变量")
print(my_model.trainable_variables)

print("\n==================================================================\n")

class MyModelWithLayerBuild(tf.keras.Model):
    def __init__(self):
        super(MyModelWithLayerBuild,self).__init__()
        self.dense_layer1=tf.keras.layers.Dense(units=2)
        self.dense_layer1.build(tf.TensorShape((None,10)))
        self.dense_layer2=tf.keras.layers.Dense(units=2)
        self.dense_layer2.build(tf.TensorShape((None,2)))
        self.a_useless_var=tf.Variable([1.1])

    def call(self, inputs, training=None, mask=None):
        output=self.dense_layer1(inputs)
        output=self.dense_layer2(output)
        return output

print("该模型也没有复写build方法，只覆盖了call方法，但不同的是在定义layer之后手动对各个layer进行了build")
my_model=MyModelWithLayerBuild()
print("调用call之前，可以看到trainable_variables有各个layer的变量")
print(my_model.trainable_variables)

print("\n==================================================================\n")

class MyModelWithModelBuild(tf.keras.Model):
    def __init__(self):
        super(MyModelWithModelBuild,self).__init__()
        self.dense_layer1=tf.keras.layers.Dense(units=2)
        self.dense_layer2=tf.keras.layers.Dense(units=2)

        self.my_layers=list()
        self.my_layers.append(tf.keras.layers.Dense(units=1)) # 这个layer是由一个list持有的
        self.a_useless_var=tf.Variable([1.1])

    def build(self, input_shape):
        # 注意build的写法 可以按注释的写法
        # self.dense_layer1.build(input_shape)
        # output_shape=self.dense_layer1.compute_output_shape(input_shape)
        # self.dense_layer2.build(output_shape)
        # output_shape=self.dense_layer2.compute_output_shape(output_shape)
        # self.my_layers[0].build(output_shape)
        # self.built=True # 构建完成后需要手动把built改为True 否则无法获取trainable_variables

        # 也可以直接这样做
        super(MyModelWithModelBuild,self).build(input_shape)

    def call(self, inputs, training=None, mask=None):
        output=self.dense_layer1(inputs)
        output=self.dense_layer2(output)
        output=self.my_layers[0](output)
        return output

print("该模型也有复写了build方法")
my_model=MyModelWithModelBuild()
print("调用build之前，无法调用trainable_variables，会报错")
# print(my_model.trainable_variables)

print("\n调用build之后，trainable_variables会返回build内构建的所有variables")
my_model.build(tf.TensorShape([None,10]))
print(my_model.trainable_variables)

该模型没有复写build方法，只覆盖了call方法
调用call之前，可以看到trainable_variables只有一个a_useless_var
[<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([1.1], dtype=float32)>]

调用call之后，可以看到trainable_variables新增了各个layer的变量
[<tf.Variable 'my_model_without_build_37/dense_185/kernel:0' shape=(10, 2) dtype=float32, numpy=
array([[-0.2475313 ,  0.442563  ],
       [-0.5502561 ,  0.18724006],
       [-0.01273036, -0.62699217],
       [ 0.66766125,  0.6239584 ],
       [ 0.6059777 , -0.31964236],
       [-0.02190101,  0.3525204 ],
       [-0.1931231 ,  0.3537925 ],
       [-0.532853  ,  0.22516954],
       [ 0.11311221, -0.3000038 ],
       [ 0.26318228,  0.2845406 ]], dtype=float32)>, <tf.Variable 'my_model_without_build_37/dense_185/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>, <tf.Variable 'my_model_without_build_37/dense_186/kernel:0' shape=(2, 2) dtype=float32, numpy=
array([[ 1.0470425 , -0.43634552],
       [-0.48062587, -0.15270066]], dtype=float32)>, <tf.Variable 'my_

综上，那么Model是如何持有所有的trainable variables的呢？

结论：在Model对象内创建Variable，都会被这个Model对象捕获

例如上面的a_useless_var，在__init__里创建的，被model捕获到了

而各个layer只有在被call了或者build之后才会创建变量。